In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/joeday/Documents/Fantasy/')

from config import INPUT_DATA_LOC, OUTPUT_DATA_LOC, MIN_YR, MAX_YR, NEEDED_COLUMNS, MAX_WEEK

In [2]:
game_week_map = pd.read_csv(OUTPUT_DATA_LOC + 'game_week_map.csv')
fantasy_plays = pd.read_csv(OUTPUT_DATA_LOC + 'fantasy_plays.csv')
weeks_played = pd.read_csv(OUTPUT_DATA_LOC + 'weeks_played.csv')
player_id_map = pd.read_csv(OUTPUT_DATA_LOC + 'player_id_map.csv')
position_map = pd.read_csv(OUTPUT_DATA_LOC + 'position_map.csv')
roster_map = pd.read_csv(OUTPUT_DATA_LOC + 'roster_map.csv')

/Users/joeday/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Team Rushing Stats

### Rushes per Team by Season

In [23]:
rushing_plays = fantasy_plays[fantasy_plays['play_type'] == 'run']
total_rushes_by_team = rushing_plays.groupby(['posteam', 'season'])['play_id'].count().reset_index()
total_rushes_by_team.columns = ['posteam', 'season', 'team_carries']
total_rushes_by_team['team_carries_per_game'] = total_rushes_by_team['team_carries'] / (MAX_WEEK-1)

In [24]:
total_rushes_by_team

,posteam,season,team_carries,team_carries_per_game
0,ARI,2009,331,22.066667
1,ARI,2010,300,20.000000
2,ARI,2011,349,23.266667
3,ARI,2012,332,22.133333
4,ARI,2013,382,25.466667
...,...,...,...,...
348,WAS,2015,382,25.466667
349,WAS,2016,353,23.533333
350,WAS,2017,376,25.066667
351,WAS,2018,395,26.333333


### Carries per Game per Player by Season

In [25]:
rushing_plays = fantasy_plays[fantasy_plays['play_type'] == 'run']
total_rushes_by_player = rushing_plays.\
                                groupby(['rusher_player_id', 'posteam', 'season', 'game_id'])['play_id'].\
                                count().\
                                reset_index()

total_rushes_by_player.columns = ['rusher_player_id',
                                  'posteam',
                                  'season',
                                  'game_id',
                                  'carries']

total_rushes_by_player = total_rushes_by_player.sort_values(by='rusher_player_id')

total_rushes_by_player_by_season = total_rushes_by_player.\
                                        groupby(['season',
                                                 'rusher_player_id',
                                                 'posteam'])\
                                        ['carries'].\
                                        sum().\
                                        reset_index()

player_carries_per_game = total_rushes_by_player_by_season.\
                            merge(weeks_played,
                                  how='inner',
                                  left_on=['rusher_player_id','season'],
                                  right_on=['player_id','season'])

player_carries_per_game['carries_per_game'] =\
    player_carries_per_game['carries'] / \
    player_carries_per_game['games_played']

player_carries_per_game.sort_values(by=['carries_per_game'],
                                ascending=False,
                                inplace=True)

player_carries_per_game = player_carries_per_game[['rusher_player_id',
                                                   'season',
                                                   'posteam',
                                                   'carries',
                                                   'games_played',
                                                   'carries_per_game']]

player_carries_per_game

,rusher_player_id,season,posteam,carries,games_played,carries_per_game
1430,00-0028198,2013,MIN,30,1,30.000000
1671,00-0028009,2014,DAL,373,15,24.866667
2704,00-0033045,2017,DAL,215,9,23.888889
117,00-0023439,2009,CIN,301,13,23.153846
1067,00-0026796,2012,HOU,335,15,22.333333
...,...,...,...,...,...,...
1976,00-0029205,2015,NO,1,4,0.250000
2801,00-0027891,2018,PHI,1,4,0.250000
1086,00-0027076,2012,JAC,1,6,0.166667
2532,00-0029205,2017,NYJ,1,7,0.142857


### RB Workload PCT

In [27]:
rb_workload_pct =  player_carries_per_game.merge(total_rushes_by_team,
                                                 how='left',
                                                 on=['season', 'posteam'])

In [29]:
rb_workload_pct['season_workload_pct'] = rb_workload_pct['carries'] / rb_workload_pct['team_carries']
rb_workload_pct['by_game_workload_pct'] = rb_workload_pct['carries_per_game'] / rb_workload_pct['team_carries_per_game']

In [31]:
rb_workload_pct[['rusher_player_id', 'season', 'posteam', 'season_workload_pct', 'by_game_workload_pct']]

,rusher_player_id,season,posteam,season_workload_pct,by_game_workload_pct
0,00-0028198,2013,MIN,0.075377,1.130653
1,00-0028009,2014,DAL,0.797009,0.797009
2,00-0033045,2017,DAL,0.487528,0.812547
3,00-0023439,2009,CIN,0.631027,0.728108
4,00-0026796,2012,HOU,0.708245,0.708245
...,...,...,...,...,...
3404,00-0029205,2015,NO,0.002770,0.010388
3405,00-0027891,2018,PHI,0.002786,0.010446
3406,00-0027076,2012,JAC,0.003012,0.007530
3407,00-0029205,2017,NYJ,0.002532,0.005425
